In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('listings.csv', encoding="latin1")

C:\Users\farhan\AppData\Local\Temp\ipykernel_7696\3422242785.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('listings.csv', encoding="latin1")


In [15]:
missing = df.isnull().sum()

In [ ]:
print(missing)

In [39]:
schema = pd.DataFrame({
    'Column': df.columns,
    'Data Type': df.dtypes.values,
    'Missing Values': df.isnull().sum().values,
    'Unique Values': df.nunique().values
})

print(schema)


                                 Column       Data Type  Missing Values  \
0                            listing_id           int64               0   
1                                  name          object               0   
2                               host_id           int64               0   
3                            host_since  datetime64[ns]               0   
4                         host_location          object               0   
5                     host_is_superhost          object               0   
6             host_total_listings_count         float64               0   
7                         neighbourhood          object               0   
8                                  city          object               0   
9                              latitude         float64               0   
10                            longitude         float64               0   
11                        property_type          object               0   
12                       

In [ ]:

for col in df.columns:
    print(col)


In [14]:
df['name'] = df['name'].fillna("Unknown")

In [ ]:

# Replaced missing Host_Location fields with listing’s city Values 
df['host_location'] = df['host_location'].fillna(df['city'])


In [22]:
import pandas as pd

# Convert host_since to datetime
df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')

# Check result
print(df['host_since'].head())
print(df['host_since'].dtypes)


0   2011-12-03
1   2013-11-29
2   2014-07-31
3   2013-12-17
4   2014-12-14
Name: host_since, dtype: datetime64[ns]
datetime64[ns]


In [24]:
# Handling missing host_since values

# 2. Fill missing with earliest known date per host_id
df['host_since'] = df.groupby('host_id')['host_since']\
                     .transform(lambda x: x.fillna(x.min()))

# 3. Still missing? Fill with median date OR mark as "Unknown"
# Option A: fill with median date across dataset
median_date = df['host_since'].median()
df['host_since'] = df['host_since'].fillna(median_date)

# Option B (alternative): create categorical "Unknown" for dashboard use
# df['host_since_filled'] = df['host_since'].fillna(pd.NaT)   # keep date column
# df['host_since_category'] = df['host_since_filled'].dt.year.fillna('Unknown')


In [26]:
# Drop selected columns
df = df.drop(['host_response_time', 'host_response_rate', 'host_acceptance_rate'], axis=1)

# Verify
print(df.columns)


Index(['listing_id', 'name', 'host_id', 'host_since', 'host_location',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'district', 'city', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bedrooms', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable'],
      dtype='object')


In [28]:
# Categorical column with missing values

df['host_is_superhost'] = df['host_is_superhost'].fillna("False")

# Numerical column with missing values

df['host_total_listings_count'] = df['host_total_listings_count'].fillna(0)



In [30]:
# Drop selected columns
df = df.drop(
    ['host_has_profile_pic', 'host_identity_verified', 'district'],
    axis=1
)

# Verify remaining columns
print(df.columns)


Index(['listing_id', 'name', 'host_id', 'host_since', 'host_location',
       'host_is_superhost', 'host_total_listings_count', 'neighbourhood',
       'city', 'latitude', 'longitude', 'property_type', 'room_type',
       'accommodates', 'bedrooms', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable'],
      dtype='object')


In [34]:
import numpy as np

df['bedrooms'] = df['bedrooms'].fillna(
    np.maximum(1, np.floor(df['accommodates'] / 2)).astype(int)
)



In [36]:
# Create a derived flag for whether a listing has any reviews
df['has_review'] = df['review_scores_rating'].notnull().astype(int)

# (Optional) Check counts
print(df['has_review'].value_counts())

has_review
1    188307
0     91405
Name: count, dtype: int64


In [38]:
# Replace missing values for review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value

# Original numeric review scores (keep NaN for math/stats)
review_cols = [
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

# Create display versions with "No reviews yet"
for col in review_cols:
    df[col + '_display'] = df[col].fillna("No reviews yet")


In [40]:
# Save refined dataset to CSV
df.to_csv("airbnb_refined.csv", index=False, encoding="utf-8")


In [ ]:
# Make a deep copy of df
df_copy = df.copy()

# Verify
print(df_copy.head())


In [43]:
# Check first 5 rows
print(df_copy['amenities'].head())

0    ["Heating", "Kitchen", "Washer", "Wifi", "Long...
1    ["Shampoo", "Heating", "Kitchen", "Essentials"...
2    ["Heating", "TV", "Kitchen", "Washer", "Wifi",...
3    ["Heating", "TV", "Kitchen", "Wifi", "Long ter...
4    ["Heating", "TV", "Kitchen", "Essentials", "Ha...
Name: amenities, dtype: object


In [45]:
# Check type and number of unique raw entries
print(df_copy['amenities'].dtype)
print(df_copy['amenities'].nunique())

object
245003


In [46]:
import ast

def parse_amenities(x):
    if isinstance(x, str):
        return [a.strip() for a in ast.literal_eval(x)]
    else:
        return []

df_copy['amenities_list'] = df_copy['amenities'].apply(parse_amenities)


In [47]:
# Flatten the list of lists into a single list
all_amenities = [amenity for sublist in df_copy['amenities_list'] for amenity in sublist]

# Get unique values
unique_amenities = set(all_amenities)
print(unique_amenities)
print(f"Total unique amenities: {len(unique_amenities)}")


{'Pantene and Head & Shoulders shampoo', 'Eletrolux aço 5 bocas stainless steel gas stove', 'Samsung with water tap unit outside refrigerator', 'Electrolux gas stove', 'DEFFY  stainless steel oven', "L'occitane conditioner", 'Radiant heating', 'brastemp cook top gas stove', 'Brstemp com congelador  refrigerator', 'Ge refrigerator', 'Breakfast available — €10 per person per day', 'Moonhaven body soap', '55" HDTV with Netflix, HBO Max, Amazon Prime Video, premium cable', 'Breakfast buffet available for a fee', 'Gliss shampoo', 'Wifi – 8 Mbps', 'Plaque de cuisson oven', '42" HDTV with Netflix, Amazon Prime Video', 'Granado shampoo', 'Clothing storage: closet', '40" HDTV with Netflix, standard cable', 'Head & Shoulder shampoo', 'Bose Revolve.  Bluetooth sound system', 'electrolux stainless steel oven', 'Hammam', 'Dove/Nivea body soap', 'marantz Amp, mission speakers sound system with aux', 'Member Mark. Sams body soap', 'Nubian Heritage Black Soap body soap', '28" HDTV with Netflix', 'Dove

In [ ]:
from collections import Counter

amenity_counts = Counter(all_amenities)
# Most common 10 amenities
print(amenity_counts.most_common(10))


[('Wifi', 260090), ('Essentials', 253532), ('Long term stays allowed', 241054), ('Kitchen', 240923), ('TV', 213037), ('Hangers', 211356), ('Hair dryer', 188724), ('Iron', 187756), ('Washer', 185073), ('Heating', 184327), ('Dedicated workspace', 179267), ('Shampoo', 174082), ('Hot water', 165163), ('Smoke alarm', 156467), ('Air conditioning', 142693), ('Dishes and silverware', 123394), ('Refrigerator', 123259), ('Cooking basics', 110255), ('Elevator', 101582), ('Bed linens', 100486)]


In [ ]:
from collections import Counter

all_amenities = [amenity for sublist in df_copy['amenities_list'] for amenity in sublist]
amenity_counts = Counter(all_amenities)

# Top 10 amenities
top_amenities = [amenity for amenity, count in amenity_counts.most_common(5)]
print(top_amenities)


['Wifi', 'Essentials', 'Long term stays allowed', 'Kitchen', 'TV', 'Hangers', 'Hair dryer', 'Iron', 'Washer', 'Heating', 'Dedicated workspace', 'Shampoo', 'Hot water', 'Smoke alarm', 'Air conditioning', 'Dishes and silverware', 'Refrigerator', 'Cooking basics', 'Elevator', 'Bed linens', 'Microwave', 'Fire extinguisher', 'Dryer', 'Stove', 'Coffee maker']


In [56]:
for amenity in top_amenities:
    df_copy[f'has_{amenity.lower().replace(" ", "_")}'] = df_copy['amenities_list'].apply(lambda x: 1 if amenity in x else 0)


In [58]:

df_copy['num_top_amenities'] = df_copy[[f'has_{amenity.lower().replace(" ", "_")}' for amenity in top_amenities]].sum(axis=1)


In [59]:
df_copy.head()

,listing_id,name,host_id,host_since,host_location,host_is_superhost,host_total_listings_count,neighbourhood,city,latitude,...,has_refrigerator,has_cooking_basics,has_elevator,has_bed_linens,has_microwave,has_fire_extinguisher,has_dryer,has_stove,has_coffee_maker,num_top_amenities
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",f,1.0,Buttes-Montmartre,Paris,48.88668,...,0,0,0,0,0,0,0,0,0,5
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",f,1.0,Buttes-Montmartre,Paris,48.88617,...,0,0,0,0,0,0,1,0,0,8
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",f,1.0,Elysee,Paris,48.88112,...,0,0,0,0,0,0,0,0,0,6
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",f,1.0,Vaugirard,Paris,48.84571,...,0,0,0,0,0,0,0,0,0,5
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",f,1.0,Passy,Paris,48.85500,...,0,0,1,0,0,0,1,0,0,10


In [60]:
# Identify binary amenity columns (columns starting with "has_")
binary_amenity_cols = [col for col in df_copy.columns if col.startswith('has_')]

# Keep only listing_id, host_id, and binary amenity columns
df_trimmed = df_copy[['listing_id', 'host_id'] + binary_amenity_cols]

# Verify
print(df_trimmed.head())
print(df_trimmed.columns)


   listing_id   host_id  has_review  has_wifi  has_essentials  \
0      281420   1466919           1         1               0   
1     3705183  10328771           1         1               1   
2     4082273  19252768           1         1               0   
3     4797344  10668311           1         1               0   
4     4823489  24837558           1         1               1   

   has_long_term_stays_allowed  has_kitchen  has_tv  has_hangers  \
0                            1            1       0            0   
1                            1            1       0            0   
2                            1            1       1            0   
3                            1            1       1            0   
4                            1            1       1            0   

   has_hair_dryer  ...  has_dishes_and_silverware  has_refrigerator  \
0               0  ...                          0                 0   
1               0  ...                          0         

In [62]:
df_trimmed.head()

,listing_id,host_id,has_review,has_wifi,has_essentials,has_long_term_stays_allowed,has_kitchen,has_tv,has_hangers,has_hair_dryer,...,has_dishes_and_silverware,has_refrigerator,has_cooking_basics,has_elevator,has_bed_linens,has_microwave,has_fire_extinguisher,has_dryer,has_stove,has_coffee_maker
0,281420,1466919,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3705183,10328771,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4082273,19252768,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4797344,10668311,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4823489,24837558,1,1,1,1,1,1,0,1,...,0,0,0,1,0,0,0,1,0,0


In [63]:
# Export the trimmed DataFrame to CSV
df_trimmed.to_csv("airbnb_amenities.csv", index=False, encoding="utf-8")

In [ ]:
import pandas as pd
import pycountry
import re

# Load only the host_location column
df_location = pd.read_csv("listings.csv", usecols=['listing_id','host_location'], encoding='latin1')

# Build dictionary of country names
countries = {c.name.lower(): c.name for c in pycountry.countries}
country_names = list(countries.keys())

# Normalize abbreviation map (all lowercase)
abbr_map = {
    "usa": "United States",
    "us": "United States",
    "uk": "United Kingdom",
    "uae": "United Arab Emirates",
    "au": "Australia",
    "fr": "France",
    "de": "Germany",
    "ie": "Ireland",
    "ca": "Canada",
    "rs": "Serbia",
    "nz": "New Zealand",
    "ru": "Russia",
    "za": "South Africa",
    "br": "Brazil",
    "in": "India",
    "ar": "Argentina",
    "mx": "Mexico",
    "ae": "Egypt",
    "ai": "Anguilla",
    "be": "Belgium",
    "al": "Albania",
    "at": "Austria",
    "ch": "Switzerland",
    "cy": "Cyprus",
    "dk": "Denmark",
    "es": "Spain",  
    "fi": "Finland",
    "gr": "Greece",
    "hr": "Croatia",
    "aus": "Australia",
    "az": "Azerbaijan", 
    "ba": "Bosnia and Herzegovina",
    "bg": "Bulgaria",   
    "cl": "Chile",
    "co": "Colombia",   
    "cr": "Costa Rica",
    "cz": "Czechia",
    "hu": "Hungary",
    "bo": "Bolivia",
    "gt": "Guatemala",
    "bs": "Bahamas",
    "by": "Belarus",
    "ec": "Ecuador",
    "cn": "China",
    "cu": "Cuba",
    "hk": "Hong Kong",
    "hksar": "Hong Kong",
    "hn": "Honduras",
    "id": "Indonesia",
    "il": "Israel",   
    "iq": "Iraq",
    "it": "Italy",
    "jo": "Jordan",
    "jm": "Jamaica",
    "jp": "Japan",
    "ke": "Kenya",
    "kr": "South Korea",
    "kw": "Kuwait",
    "kz": "Kazakhstan",
    "lb": "Lebanon",
    "lt": "Lithuania",
    "lu": "Luxembourg",
    "ly": "Libya",
    "ma": "Morocco",
    "md": "Moldova",
    "mex": "Mexico",
    "nl": "Netherlands",
    "no": "Norway",
    "np": "Nepal",
    "om": "Oman",
    "pe": "Peru",
    "ph": "Philippines",
    "pk": "Pakistan",
    "pl": "Poland",
    "pt": "Portugal",
    "qa": "Qatar",
    "ro": "Romania",
    "rs": "Serbia",
    "rw": "Rwanda",
    "se": "Sweden",
    "sg": "Singapore",
    "si": "Slovenia",
    "sk": "Slovakia",
    "th": "Thailand",
    "tn": "Tunisia",
    "tw": "Taiwan",
    "tz": "Tanzania",   
    "ua": "Ukraine",
    "ug": "Uganda",
    "uy": "Uruguay",
    "uz": "Uzbekistan",
    "vn": "Vietnam",
    "za": "South Africa",
    "zm": "Zambia",
    "zw": "Zimbabwe"

}

def extract_country(text):
    if pd.isna(text):
        return None
    
    # Clean text (remove punctuation/numbers, lowercase)
    text_clean = re.sub(r"[^a-zA-Z ]", " ", str(text)).lower()
    
    # Check abbreviations first
    for abbr, full_name in abbr_map.items():
        if re.search(rf"\b{abbr}\b", text_clean):
            return full_name
    
    # Check official country names
    for name in country_names:
        if name in text_clean:
            return countries[name]
    
    return "Unknown"

# Apply function
df_location["country"] = df_location["host_location"].apply(extract_country)

# Preview
print(df_location.head(20))


    listing_id                 host_location  country
0       281420  Paris, Ile-de-France, France  Germany
1      3705183  Paris, Ile-de-France, France  Germany
2      4082273  Paris, Ile-de-France, France  Germany
3      4797344  Paris, Ile-de-France, France  Germany
4      4823489  Paris, Ile-de-France, France  Germany
5      4898654  Paris, Ile-de-France, France  Germany
6      6021700  Paris, Ile-de-France, France  Germany
7      6945740  Paris, Ile-de-France, France  Germany
8      7491966  Paris, Ile-de-France, France  Germany
9      7849932  Paris, Ile-de-France, France  Germany
10     7995091  Paris, Ile-de-France, France  Germany
11     9011901  Paris, Ile-de-France, France  Germany
12     9880847  Paris, Ile-de-France, France  Germany
13     9898855  Paris, Ile-de-France, France  Germany
14    10266213  Paris, Ile-de-France, France  Germany
15    11643413  Paris, Ile-de-France, France  Germany
16    11764434  Paris, Ile-de-France, France  Germany
17    11963112  Paris, Ile-d

In [19]:
# Get unique countries
unique_countries = df_location['country'].unique()

# Print them
print(unique_countries)




['Germany' 'United States' 'France' None 'Mexico' 'Unknown' 'Tunisia'
 'Ireland' 'Serbia' 'Canada' 'South Africa' 'Brazil' 'Australia' 'Italy'
 'Argentina' 'China' 'Hong Kong' 'Netherlands' 'United Kingdom'
 'Switzerland' 'Russia' 'Thailand' 'India' 'Singapore' 'Spain' 'Belgium'
 'Romania' 'Colombia' 'Mongolia' 'Jersey' 'Israel' 'Anguilla' 'Morocco'
 'Lebanon' 'Nepal' 'Monaco' 'Cyprus' 'Denmark' 'Austria' 'Czechia'
 'Indonesia' 'Georgia' 'Luxembourg' 'Sweden' 'Kenya' 'Egypt' 'Slovenia'
 'New Zealand' 'United Arab Emirates' 'Greece' 'Iceland' 'Japan'
 'Philippines' 'Martinique' 'Jamaica' 'South Korea' 'Portugal' 'Poland'
 'Guinea' 'Guadeloupe' 'Norway' 'Estonia' 'Malaysia' 'Costa Rica' 'Jordan'
 'Croatia' 'French Guiana' 'Peru' 'Chile' 'Belarus' 'Bulgaria' 'Lithuania'
 'Slovakia' 'Saudi Arabia' 'Cambodia' 'French Polynesia' 'Ukraine'
 'Albania' 'Kuwait' 'Hungary' 'Panama' 'Bosnia and Herzegovina'
 'Puerto Rico' 'Malta' 'Ecuador' 'Taiwan' 'Finland' 'Oman' 'Mayotte'
 'Algeria' 'Pakistan' 

In [31]:
# Save df_location to CSV
df_location.to_csv("df_location_with_country.csv", index=False, encoding='utf-8')

print("File saved as df_location_with_country.csv")


File saved as df_location_with_country.csv


In [13]:
# Get unique countries
unique_Location = df_location['host_location'].unique()

# Print them
print(unique_Location)


['Paris, Ile-de-France, France' 'New York, New York, United States'
 'Hericy, Ile-de-France, France' ...
 'Saint-Jean-de-Fos, Occitanie, France'
 'Lindfield, England, United Kingdom'
 'BrasÃ\x83Â\xadlia de Minas, State of Minas Gerais, Brazil']


In [30]:
# Count Unknown values
unknown_count = (df_location["country"] == "Unknown").sum()

print("Number of Unknown countries:", unknown_count)

# If you also want percentage
total = len(df_location)
print("Percentage Unknown:", round((unknown_count/total)*100, 2), "%")


Number of Unknown countries: 0
Percentage Unknown: 0.0 %


In [23]:
# Filter rows where country extraction failed
unknown_rows = df_location[df_location["country"] == "Unknown"]

# Show first 20 raw host_location values that failed
print("Sample of host_location with Unknown country:")
print(unknown_rows["host_location"].head(20).to_list())

# Count how many unique unknown locations there are
unique_unknowns = unknown_rows["host_location"].nunique()
print("\nUnique Unknown host_location entries:", unique_unknowns)

# Show top 20 most frequent unknown locations
print("\nMost common Unknown host_location values:")
print(unknown_rows["host_location"].value_counts().head(20))


Sample of host_location with Unknown country:
['Nizhny Novgorod, Nizhny Novgorod Oblast, Russia', 'Taoyuan City, Taiwan', 'Brooklyn, NY', 'Brooklyn, NY', 'Manhattan', 'SarÃ\x84Â±yer, Istanbul, Turkey', 'London, England', 'Bangkok', 'Ho Chi Minh City, Ho Chi Minh, Vietnam', 'Manhattan, NY', 'Brasil', "I'm residing between Paris and Barcelona", 'RE', 'Moscow, Moscow, Russia', 'SA', 'roma vaticano', 'Brasil', 'Maroubra Beach', 'BÃ\x83Â¼yÃ\x83Â¼kcekmece, Istanbul, Turkey', 'Italia']

Unique Unknown host_location entries: 695

Most common Unknown host_location values:
host_location
Bangkok                           418
Fethiye, MuÃÅ¸la, Turkey         189
Head of Innovation & Revenue      107
Brasil                            102
Ã©Â¦â¢Ã¦Â¸Â¯                      76
London                             74
Ankara, Ankara, Turkey             40
Brooklyn, NY                       40
Bahcelievler, Istanbul, Turkey     40
Bangkok /LONDON                    39
Amministratore Unico               3

In [26]:
def replace_unknowns(df, replacements):
    """
    Replace Unknown countries in df['country'] 
    using a dictionary of replacements.

    Parameters:
    df (DataFrame): The DataFrame containing 'country' column.
    replacements (dict): { "raw_host_location_value": "Correct Country" }
    """
    df['country'] = df.apply(
        lambda row: replacements.get(row['host_location'], row['country']),
        axis=1
    )
    return df


# Example usage:
manual_replacements = {
    "Istanbul, Istanbul, Turkey": "Turkey",
    "TR": "Turkey",
    "Istanbul, Turkey": "Turkey",
    "Istanbul": "Turkey",  # If context-specific
    "ÃÅ¾iÃÅ¸li, Istanbul, Turkey": "Turkey",
    "Bangkok, Thailand": "Thailand",
    "GB": "United Kingdom",
    "Turkey": "Turkey",
    "Fatih, Istanbul, Turkey": "Turkey",
    "Fethiye, MuÃÅ¸la, Turkey ": "Turkey",
    "Istanbul, Istanbul Province, Turkey": "Turkey",
    "KadÃÂ±kÃÂ¶y, Istanbul, Turkey": "Turkey",
    "Brasilia, Brazil": "Brazil",
    "MY": "Malaysia",
    "London, England, United Kingdom": "United Kingdom",
    "Izmir, ÃÂ°zmir, Turkey": "Turkey",
    "BeylikdÃÂ¼zÃÂ¼ Osb, Istanbul, Turkey": "Turkey",
    "CX": "Christmas Island",
    "Bangkok": "Thailand",
    "Fethiye, MuÃÅ¸la, Turkey": "Turkey",
    "Brasil": "Brazil",
    "London": "United Kingdom",
    "Ankara, Ankara, Turkey": "Turkey",
    "Brooklyn, NY": "United States",
    "Bahcelievler, Istanbul, Turkey": "Turkey",
    "Moscow, Moscow, Russia": "Russia",
    "Italia": "Italy",
    "AtaÃÅ¸ehir, Istanbul, Turkey": "Turkey",
    "ÃÅ¾ile, Istanbul, Turkey": "Turkey",
    "Istanbul, TR": "Turkey",
    "Istanbul / Turkey": "Turkey",
    "Pendik, Istanbul, Turkey": "Turkey",
    "istanbul tÃÂ¼rkiye": "Turkey",
    "Istanbul,Turkey": "Turkey",

}

df_location = replace_unknowns(df_location, manual_replacements)


In [29]:
# Filter rows where country extraction failed
unknown_rows = df_location[df_location["country"] == "Unknown"]

# Show first 20 raw host_location values that failed
print("Sample of host_location with Unknown country:")
print(unknown_rows["host_location"].head(20).to_list())

# Count how many unique unknown locations there are
unique_unknowns = unknown_rows["host_location"].nunique()
print("\nUnique Unknown host_location entries:", unique_unknowns)

# Show top 20 most frequent unknown locations
print("\nMost common Unknown host_location values:")
print(unknown_rows["host_location"].value_counts().head(20))

Sample of host_location with Unknown country:
[]

Unique Unknown host_location entries: 0

Most common Unknown host_location values:
Series([], Name: count, dtype: int64)


In [28]:
# Drop rows where country is Unknown
df_location = df_location[df_location['country'] != "Unknown"].copy()

# Reset index if needed
df_location.reset_index(drop=True, inplace=True)


In [35]:
import pandas as pd

# Load only listing_id and property_type columns
df_property = pd.read_csv("listings.csv", usecols=["listing_id", "property_type"], encoding="utf-8")

print(df_property.head())


   listing_id     property_type
0      281420  Entire apartment
1     3705183  Entire apartment
2     4082273  Entire apartment
3     4797344  Entire apartment
4     4823489  Entire apartment


In [36]:
distinct_count = df["property_type"].nunique()
print(f"Number of distinct property types: {distinct_count}")


Number of distinct property types: 144


In [37]:
property_counts = df["property_type"].value_counts()

print("Property type counts:")
print(property_counts)


Property type counts:
property_type
Entire apartment                138989
Private room in apartment        47322
Private room in house            13292
Entire house                     13273
Entire condominium               11250
                                 ...  
Igloo                                1
Private room in cave                 1
Holiday park                         1
Private room in holiday park         1
Tipi                                 1
Name: count, Length: 144, dtype: int64


In [ ]:
import pandas as pd

# remove display limit
pd.set_option("display.max_rows", None)

print(df_property)


In [39]:
print(df_property["property_type"].value_counts())


property_type
Entire apartment                      138989
Private room in apartment              47322
Private room in house                  13292
Entire house                           13273
Entire condominium                     11250
Room in boutique hotel                  5771
Entire loft                             4587
Private room in condominium             4462
Private room in bed and breakfast       4238
Entire serviced apartment               3973
Room in hotel                           3205
Private room in townhouse               2959
Shared room in apartment                2420
Entire townhouse                        2331
Entire guest suite                      2273
Private room in serviced apartment      1542
Entire villa                            1508
Room in aparthotel                      1495
Private room in guest suite             1370
Private room in guesthouse              1337
Private room in hostel                  1232
Entire guesthouse                       1

In [40]:

# Define mapping function
def categorize_property(ptype):
    ptype = str(ptype).lower()
    
    # Apartments & Condos
    if any(x in ptype for x in ["apartment", "condominium", "loft", "flat", "floor", "studio"]):
        return "Apartment/Condo"
    
    # Houses
    elif any(x in ptype for x in ["house", "townhouse", "villa", "cottage", "bungalow", "cabin", "chalet", "tiny house"]):
        return "House"
    
    # Hotels & Resorts
    elif any(x in ptype for x in ["hotel", "resort", "serviced apartment", "aparthotel", "boutique hotel", "pension", "heritage hotel"]):
        return "Hotel/Resort"
    
    # Guesthouses & BnBs
    elif any(x in ptype for x in ["guesthouse", "guest suite", "bed and breakfast", "bnb", "pousada", "holiday park"]):
        return "Guesthouse/BnB"
    
    # Unique stays
    elif any(x in ptype for x in ["farm stay", "treehouse", "yurt", "dome", "castle", "island", "boat", "houseboat",
                                  "train", "rv", "campsite", "tent", "hut", "cave", "barn", "igloo", "windmill", "lighthouse"]):
        return "Unique Stay"
    
    # Shared spaces
    elif "shared" in ptype or "dorm" in ptype:
        return "Shared Space"
    
    else:
        return "Other"

# Apply function
df_property["property_category"] = df_property["property_type"].apply(categorize_property)

# Check counts
category_counts = df_property["property_category"].value_counts()
print(category_counts)



property_category
Apartment/Condo    216711
House               39073
Hotel/Resort        10589
Guesthouse/BnB       9193
Other                2792
Shared Space          873
Unique Stay           481
Name: count, dtype: int64


In [41]:
# Save grouped dataset
df_property.to_csv("property_grouped.csv", index=False)

In [3]:
import pandas as pd

# Load only listing_id and amenities columns
df_amenities = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'amenities'], 
    encoding='latin1'  # use latin1 if utf-8 fails
)

# Preview the data
print(df_amenities.head())


   listing_id                                          amenities
0      281420  ["Heating", "Kitchen", "Washer", "Wifi", "Long...
1     3705183  ["Shampoo", "Heating", "Kitchen", "Essentials"...
2     4082273  ["Heating", "TV", "Kitchen", "Washer", "Wifi",...
3     4797344  ["Heating", "TV", "Kitchen", "Wifi", "Long ter...
4     4823489  ["Heating", "TV", "Kitchen", "Essentials", "Ha...


In [4]:
# Save the dataframe to CSV
df_amenities.to_csv("listing_amenities.csv", index=False, encoding='utf-8')

print("CSV file saved successfully!")


CSV file saved successfully!


In [2]:
import pandas as pd

# Load only listing_id and Latitude, Longitude details
df_locations = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'latitude', 'longitude', 'city'],
    encoding='latin1'  # use latin1 if utf-8 fails
)

# Preview the data
print(df_locations.head())


   listing_id   city  latitude  longitude
0      281420  Paris  48.88668    2.33343
1     3705183  Paris  48.88617    2.34515
2     4082273  Paris  48.88112    2.31712
3     4797344  Paris  48.84571    2.30584
4     4823489  Paris  48.85500    2.26979


In [ ]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from tqdm import tqdm

# Load your file
df = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'latitude', 'longitude', 'city'],
    encoding='latin1') # use latin1 if utf-8 fails

# Initialize geolocator with a custom user_agent
geolocator = Nominatim(user_agent="airbnb_project")

def get_location(lat, lon, retries=3):
    """Get city, state, country from latitude & longitude with retries"""
    for attempt in range(retries):
        try:
            location = geolocator.reverse((lat, lon), language="en", addressdetails=True, timeout=10)
            if location and "address" in location.raw:
                addr = location.raw["address"]
                return {
                    "city": addr.get("city") or addr.get("town") or addr.get("village"),
                    "state": addr.get("state"),
                    "country": addr.get("country")
                }
        except (GeocoderTimedOut, GeocoderServiceError):
            time.sleep(2)  # wait before retry
            continue
    return {"city": None, "state": None, "country": None}

# Apply function to dataframe with progress bar
results = []
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Geocoding Listings"):
    results.append(get_location(row["latitude"], row["longitude"]))

# Convert list of dicts into DataFrame
results_df = pd.DataFrame(results)

# Merge with original dataframe
df_location = pd.concat([df, results_df], axis=1)

# Save to new file
df_location.to_csv("listings_with_location.csv", index=False)
 
print("✅ Done! File saved as listings_with_location.csv")


Geocoding Listings:  11%|█▏        | 32086/279712 [11:18:29<68:31:26,  1.00it/s]   

In [2]:
import pandas as pd

df = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'host_location', 'city'],
    encoding='latin1') # use latin1 if utf-8 fails

In [3]:
df.to_csv("listings_with_location.csv", index=False)

In [7]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from tqdm import tqdm

# Load only the first 100 rows
df = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'host_location', 'latitude', 'longitude'],
    encoding='latin1').head(100)

# Initialize geolocator with a custom user_agent
geolocator = Nominatim(user_agent="airbnb_project")

def get_location(lat, lon, retries=3):
    """Get city, state, country from latitude & longitude with retries"""
    for attempt in range(retries):
        try:
            location = geolocator.reverse(
                (lat, lon),
                language="en",
                addressdetails=True,
                timeout=10
            )
            if location and "address" in location.raw:
                addr = location.raw["address"]
                return {
                    "city": addr.get("city") or addr.get("town") or addr.get("village"),
                    "state": addr.get("state"),
                    "country": addr.get("country")
                }
        except (GeocoderTimedOut, GeocoderServiceError):
            time.sleep(2)  # wait before retry
            continue
    return {"city": None, "state": None, "country": None}

# Apply function to dataframe with progress bar
results = []
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Geocoding First 100 Listings"):
    results.append(get_location(row["latitude"], row["longitude"]))

# Convert list of dicts into DataFrame
results_df = pd.DataFrame(results)

# Merge with original dataframe
df_location = pd.concat([df, results_df], axis=1)

# Save to new file
df_location.to_csv("listings_first100_with_location.csv", index=False)

print("✅ Done! File saved as listings_first100_with_location.csv")


Geocoding First 100 Listings: 100%|██████████| 100/100 [02:54<00:00,  1.74s/it]

✅ Done! File saved as listings_first100_with_location.csv


In [12]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from tqdm import tqdm
import os

# Load next n rows (skip first n)
df = pd.read_csv(
    "listings.csv", 
    usecols=['listing_id', 'host_location', 'latitude', 'longitude'],
    encoding='latin1').iloc[2000:3000]

# Initialize geolocator
geolocator = Nominatim(user_agent="airbnb_project")

def get_location(lat, lon, retries=3):
    """Get city, state, country from latitude & longitude with retries"""
    for attempt in range(retries):
        try:
            location = geolocator.reverse(
                (lat, lon),
                language="en",
                addressdetails=True,
                timeout=10
            )
            if location and "address" in location.raw:
                addr = location.raw["address"]
                return {
                    "city": addr.get("city") or addr.get("town") or addr.get("village"),
                    "state": addr.get("state"),
                    "country": addr.get("country")
                }
        except (GeocoderTimedOut, GeocoderServiceError):
            time.sleep(2)
            continue
    return {"city": None, "state": None, "country": None}

# Apply function with progress bar
results = []
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Geocoding Next 100 Listings"):
    results.append(get_location(row["latitude"], row["longitude"]))

# Convert list of dicts into DataFrame
results_df = pd.DataFrame(results)

# Merge with original dataframe slice
df_location = pd.concat([df, results_df], axis=1)

# Append to existing CSV (no header this time)
output_file = "listings_first100_with_location.csv"

if os.path.exists(output_file):
    df_location.to_csv(output_file, mode="a", index=False, header=False)  # append
else:
    df_location.to_csv(output_file, index=False)  # create if not exists

print("✅ Done! Next n rows appended to listings_first100_with_location.csv")


Geocoding Next 100 Listings: 100%|██████████| 1000/1000 [42:13<00:00,  2.53s/it]   

✅ Done! Next n rows appended to listings_first100_with_location.csv
